In [1]:
import numpy as np
import widgyts
import ipydatawidgets
import ipywidgets
import traitlets
from IPython.display import DisplayObject, display
import yt
import h5py

First we'll load IsolatedGalaxy, one of the datasets that is used throughout yt's documentation. 

### Slice from Pop II prime through the densest location 

In [2]:
# loading in the data
with h5py.File("/Users/madicken/yt_data/scipydemo/DD0182_slice.h5", "r") as f:
    s = {}
    for field in ("px", "py", "pdx", "pdy", "density", "temperature"):
        s[field] = f[field][:]
        
# putting data into a FRBViewer (loading the data into wasm)
frb_dens = widgyts.FRBViewer(height = 512, width = 512,
                            px = s["px"], py = s["py"], pdx = s["pdx"], pdy = s["pdy"],
                            val = s["density"],
                            layout = ipywidgets.Layout(width = '500px', height='500px'))

#using the default controls menu
controls = frb_dens.setup_controls()
display(ipywidgets.HBox([controls,frb_dens]))

Let's add another zoom widget with a log scale. The default uses a linear scale.

In [3]:
frb_dens.colormaps.is_log = True
width_slider = ipywidgets.FloatLogSlider(value = 0.0, max = 0.0, min = np.log10(512 * s["pdx"].min()), base = 10)

width_slider.value = 1.0
def on_changed(event):
    frb_dens.view_width = (event['new'], event['new'])
width_slider.observe(on_changed, 'value', 'change')
width_slider

FloatLogSlider(value=1.0, max=0.0, min=-8.428839878591473)

#### We can also link our widget traitlets to explore multiple fields at the same time. 

In [4]:
frb_temp = widgyts.FRBViewer(height = 512, width = 512,
                            px = s["px"], py = s["py"], pdx = s["pdx"], pdy = s["pdy"],
                            val = s["temperature"], layout=ipywidgets.Layout(width = '500px', height='500px'))
ipywidgets.link((frb_dens, 'view_center'), (frb_temp, 'view_center'))
ipywidgets.link((frb_dens, 'view_width'), (frb_temp, 'view_width'))
ipywidgets.link((frb_dens.colormaps, 'is_log'), (frb_temp.colormaps, 'is_log'))
display(ipywidgets.VBox([width_slider,ipywidgets.HBox([controls,frb_dens, frb_temp])]))

### We can also access the temperature field properties through the traitlets associated 
### with the widget and update them. These will set the max and min vals for the temperature 
### plot and update its colormap. 

frb_temp.colormaps.map_name='inferno'
frb_temp.colormaps.min_val=5.874986614501054
frb_temp.colormaps.max_val=15653.545441919285

### Projection of Pop II prime dataset

Instead of a slice, let's try and look at a projection of the same dataset. This will use all of the same linking features we looked at above. 

In [5]:
# reading in the projection data
with h5py.File("/Users/madicken/yt_data/scipydemo/DD0182_proj.h5", "r") as f:
    s = {}
    for field in ("px", "py", "pdx", "pdy", "density", "temperature"):
        s[field] = f[field][:]

# putting the fields into two different FRB views
frb_projd = widgyts.FRBViewer(height = 512, width = 512,
                            px = s["px"], py = s["py"], pdx = s["pdx"], pdy = s["pdy"],
                            val = s["density"], layout=ipywidgets.Layout(width = '500px', height='500px'))
frb_projt = widgyts.FRBViewer(height = 512, width = 512,
                            px = s["px"], py = s["py"], pdx = s["pdx"], pdy = s["pdy"],
                            val = s["temperature"], layout=ipywidgets.Layout(width = '500px', height='500px'))

# linking the views so their windows are aligned
ipywidgets.link((frb_projd, 'view_center'), (frb_projt, 'view_center'))
ipywidgets.link((frb_projd, 'view_width'), (frb_projt, 'view_width'))
ipywidgets.link((frb_projd.colormaps, 'is_log'), (frb_projt.colormaps, 'is_log'))

#adding in the log slider 
width_slider2 = ipywidgets.FloatLogSlider(value = 0.0, max = 0.0, min = np.log10(512 * s["pdx"].min()), base = 10)
width_slider2.value = 1.0
def on_changed(event):
    frb_projd.view_width = (event['new'], event['new'])
width_slider2.observe(on_changed, 'value', 'change')

#adding in the controls
controls = frb_projd.setup_controls()
display(ipywidgets.VBox([width_slider2,ipywidgets.HBox([controls,frb_projd, frb_projt])]))

# setting bounds of the temperature plot
frb_projt.colormaps.map_name='inferno'
frb_projt.colormaps.min_val=8.5
frb_projt.colormaps.max_val=1283

# setting the log scale
frb_projd.colormaps.is_log = True